In [2]:
import utils.connect_mods as conn
import utils.payload_construct_mods as paystruct

In [3]:
import yaml
import json

with open('device_config.yaml') as f:
    
    data = yaml.load(f, Loader=yaml.FullLoader)
    print(json.dumps(data, indent=4))

{
    "mqtt": {
        "oringpaas": {
            "client_id": "thing:syjmHv7z-",
            "host": "mqtt.paas.oringnet.cloud",
            "port": 1883,
            "username": "syjmHv7z-",
            "userpw": "ZCsC5XNCsk~mzrsy",
            "keepalive": 60,
            "topic": {
                "Command": "$thing/syjmHv7z-/$cmd/$downlink/",
                "Downlink config": "$thing/syjmHv7z-/$conf/$expected/",
                "Uplink config": "$thing/syjmHv7z-/$conf/$reported/",
                "Data": "$thing/syjmHv7z-/$data/"
            },
            "publish_path": {
                "dataBucket": "sensorData",
                "dataShapes": [
                    "voltage",
                    "cpuTemp"
                ]
            }
        }
    }
}


In [4]:
data["mqtt"]["oringpaas"]["publish_path"]["dataShapes"]

['voltage', 'cpuTemp']

In [6]:
a = conn.mqtt_connack(**data['mqtt']['oringpaas'])
a.connect()
a.payload_submit(json.dumps({"id":"voltage","value":'test'}))
a.client.disconnect()

connect to mqtt.paas.oringnet.cloud:1883
Connected with result code 0
userdata: None, mid: 1
is published: True


0

In [ ]:
import joblib
import time
import scipy.stats

a = conn.mqtt_connack(**data['mqtt']['oringpaas'])
a.connect()
a.client.loop_start()

while True:
    sim_data = {}
    cols = ['DO', 'BOD', 'SS', 'Nitrogen']
    for i in range(4):
        kde = joblib.load(f'./sim_kdes/kde{i}.pkl')
        _data = kde.resample(1).T[:,0]
        sim_data[cols[i]] = str(round(_data[0],2))
    
    a.payload_submit(json.dumps({"id":"fullDataPackt","value":str(sim_data)}))
    time.sleep(5.0)
# sim_data